In [1]:
import pandas as pd
import re
import os
from IPython.display import clear_output
import ToolBox.kit_function as kit

In [2]:
listSheet = os.listdir('Dados')
table_master = pd.DataFrame({})
columns = ['Nome','codigo','prc','imp','link']

for index,sheet in enumerate(listSheet):
    clear_output(wait=False)  
    print(f'Concatenando {sheet} na Tabela Master.\n Planilha {index+1} de {len(listSheet)}')
    aux_sheet = kit.readExcel(str('Dados/' + sheet) , columns)
    table_master = pd.concat([table_master, aux_sheet])

table_master =  table_master.reset_index(drop=True)

Concatenando VIRA MACHO.xlsx na Tabela Master.
 Planilha 127 de 127


In [3]:
def regexClean(string, regexExpression, default_value=''):
    # Compila e encontra todas as ocorrências
    result = re.compile(regexExpression).findall(string)
    
    # Se o resultado for vazio, retorna o valor padrão
    if not result:
        return default_value
    
    # Se o resultado for uma tupla, retorna o primeiro ou segundo valor da tupla
    if isinstance(result[0], tuple):
        return result[0][0] or result[0][1]
    
    # Se o resultado for uma lista simples, retorna o primeiro item
    return result[0]

In [4]:
expCodigo = r"[0-9]+" #5400455
expPrc = r"[0-9]+[,][0-9]+" #10,47

expIPI1 = r'[IP]+[: ]+[0-9]+%' #IPI: 5%
expIPI2 = r'[IP]+[: ]+[0-9]+[,][0-9]+%' #IPI: 5,5%

expST1 = r'[ST]+[: ]+[0-9]+%' #ST: 27%
expST2 = r'[ST]+[: ]+[0-9]+[,][0-9]+%' #ST: 27,68%

expPOR1 = r'[0-9]+%' #27%
expPOR2 = r'[0-9]+[,][0-9]+%' #27,68%

# Combina ambas as expressões
combined_ST = f"({expST1})|({expST2})"
combined_IPI = f"({expIPI1})|({expIPI2})"
combined_POR = f"({expPOR1})|({expPOR2})"

dataCleaned = {'Nome':[],
    'codigo': [],
    'prc': [],
    'prcAntigo': [],
    'ipi': [],
    'st': [],
    'impAntigo': [],
    'link':[]}

In [9]:
for line in range(len(table_master)):
    codigo = regexClean(table_master['codigo'][line],expCodigo)
    preco = regexClean(table_master['prc'][line],expPrc)
    ipi = regexClean(regexClean(table_master['imp'][line],combined_IPI),combined_POR)
    st = regexClean(regexClean(table_master['imp'][line],combined_ST),combined_POR)  

    row_data = {
        'Nome': table_master['Nome'][line],
        'codigo': codigo,
        'prc': preco,
        'prcAntigo': table_master['prc'][line],
        'ipi': ipi,
        'st': st,
        'impAntigo': table_master['imp'][line],
        'link': table_master['link'][line]
    }

    for key, value in row_data.items():
        dataCleaned[key].append(value)
    

In [ ]:
xlsx_dir = 'Links/'
title = 'Scrap-mtx'
mtxTable = pd.DataFrame(dataCleaned)
file_name = xlsx_dir + title + '.xlsx'
mtxTable.to_excel(file_name, index=False)
print(f'A planilha {title}.xlsx foi salva em {file_name}')

A planilha Scrap-mtx.xlsx foi salva em Links/Scrap-mtx.xlsx
